In [1]:
import sqlite3
import json
import os

# File paths
DB_NAME = "weather_archive.db"
LOG_FILE = "data_management_system_logs.txt"
UPTIME_FILE = "sensor_uptime.json"
INPUT_FILE = "wilderness_weather_data.json"
ARCHIVED_FILE = "wilderness_weather_data_old.json"

def log_message(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(message + "\n")
    print(message)

# Connect to database
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# Drop and recreate table (clean slate for testing)
cursor.execute("DROP TABLE IF EXISTS weather_data")
cursor.execute("""
CREATE TABLE weather_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    place TEXT,
    latitude REAL,
    longitude REAL,
    temperature REAL,
    humidity INTEGER,
    pressure INTEGER,
    timestamp TEXT,
    risk_level INTEGER
)
""")
conn.commit()

# Insert if not already present
def store_weather_data(data):
    cursor.execute("""
    SELECT * FROM weather_data WHERE place=? AND timestamp=?
    """, (data['place'], data['timestamp']))
    
    if cursor.fetchone() is None:
        cursor.execute("""
        INSERT INTO weather_data 
        (place, latitude, longitude, temperature, humidity, pressure, timestamp, risk_level) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            data['place'],
            data['latitude'],
            data['longitude'],
            data['temperature'],
            data['humidity'],
            data['pressure'],
            data['timestamp'],
            data.get('risk_level', 0)
        ))
        conn.commit()

        if data.get('risk_level', 0) >= 7:
            log_message(f"RED ALERT archived for {data['place']} at {data['timestamp']} (Risk Score: {data['risk_level']})")
        else:
            log_message(f"Archived weather data for {data['place']} at {data['timestamp']}.")
    else:
        log_message(f"Skipped duplicate for {data['place']} at {data['timestamp']}.")

# Show stored data
def print_archived_data():
    cursor.execute("SELECT * FROM weather_data")
    records = cursor.fetchall()
    if records:
        log_message("\nArchived Weather Data:")
        for record in records:
            log_message(str(record))
    else:
        log_message("No weather data has been archived yet.")

def process_and_archive_data():
    if not os.path.exists(INPUT_FILE):
        log_message("Input file not found. Nothing to process.")
        return

    with open(INPUT_FILE, 'r') as f:
        lines = f.readlines()

    if not lines:
        log_message("No new weather data found.")
        return

    success = True

    with open(ARCHIVED_FILE, 'a') as archive:
        for line in lines:
            try:
                data = json.loads(line)
                required_keys = ['place', 'latitude', 'longitude', 'temperature', 'humidity', 'pressure', 'timestamp']
                if all(k in data for k in required_keys):
                    store_weather_data(data)
                    archive.write(line)
                else:
                    log_message(f"Incomplete data entry: {data.get('place', 'Unknown place')}")
                    success = False
            except json.JSONDecodeError as e:
                log_message(f"JSON error: {e}")
                success = False

    if success:
        with open(INPUT_FILE, 'w') as f:
            pass  # Clear file only if successful
        log_message("All data archived successfully. Input file cleared.")
    else:
        log_message("Errors encountered. Input file retained for review.")

if __name__ == "__main__":
    process_and_archive_data()
    print_archived_data()
    conn.close()
    log_message("\nWeather data archiving complete.")


Archived weather data for Yellowstone National Park at 2025-04-28 21:27:48.
Archived weather data for Yosemite National Park at 2025-04-28 21:27:48.
Archived weather data for Grand Canyon National Park at 2025-04-28 21:27:48.
Archived weather data for Great Smoky Mountains National Park at 2025-04-28 21:27:48.
Archived weather data for Zion National Park at 2025-04-28 21:27:48.
Archived weather data for Acadia National Park at 2025-04-28 21:27:48.
Archived weather data for Everglades National Park at 2025-04-28 21:27:48.
Archived weather data for Rocky Mountain National Park at 2025-04-28 21:27:48.
Archived weather data for Yellowstone National Park at 2025-04-27 21:27:48.
Archived weather data for Yosemite National Park at 2025-04-27 21:27:48.
Archived weather data for Grand Canyon National Park at 2025-04-27 21:27:48.
Archived weather data for Great Smoky Mountains National Park at 2025-04-27 21:27:48.
Archived weather data for Zion National Park at 2025-04-27 21:27:48.
Archived weat